In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType,FloatType

In [0]:
#dbutils.fs.rm('dbfs:/FileStore/tables',True)

Out[23]: True

In [0]:
def read_file(fileformat,path,**options):
    df=spark.read.format(fileformat).options(**options).load(path)
    return df

In [0]:
def summary_df(df):
    return df.summary()

In [0]:
def null_per(df):
    schema = StructType([ \
        StructField("Column",StringType(),True), \
        StructField("Null_Percentage",StringType(),True)
    ])
    emptyRDD = spark.sparkContext.emptyRDD()
    resultdf = spark.createDataFrame([], schema=schema)
    for i in df.columns:
        df_count_null=df.select(col(i)).filter(col(i).isNull() | (col(i) == '')).count()
        df_nnul= spark.createDataFrame([[f'{i}',f"{df_count_null*100/df.count()} %" ]],schema=schema)
        resultdf=resultdf.union(df_nnul)
    return resultdf.orderBy(desc("Null_Percentage"))

In [0]:
def unique_values(df):
    resultdf = None
    for i in df.columns:
        unique_values_count = df.select(i).distinct().count()
        data = [(i, unique_values_count)]
        schema = StructType([
            StructField("Column", StringType(), True),
            StructField("UniqueValuesCount", IntegerType(), True)
        ])
        df_res = spark.createDataFrame(data, schema=schema)
        if resultdf is None:
            resultdf = df_res
        else:
            resultdf = resultdf.union(df_res)
    return resultdf.orderBy("UniqueValuesCount")

In [0]:
def count_of_unique_value(df,column):
    distinct_value_counts = df.groupBy(column).agg(count("*").alias("Count"))
    return distinct_value_counts.orderBy(desc("Count"))

In [0]:
def miss_matched(df,Dictt):
    schema = StructType([ \
        StructField("Column",StringType(),True), \
        StructField("MismatchPercentage",StringType(),True)
    ])
    resultdf = spark.createDataFrame([], schema=schema)
    for i,y in Dictt.items():
        df_regex_not_like_count = df_1.select(col(i)).filter(~col(i).rlike(y)).count()
        df_for_loop= spark.createDataFrame([[f'{i}',f"{df_regex_not_like_count*100/df.count()} %" ]],schema=schema)
        resultdf=resultdf.union(df_for_loop)
    return resultdf.orderBy(desc("MismatchPercentage"))

Column,MismatchPercentage
age,7.5 %
gender,7.5 %
first_name,0.0 %


In [0]:
schema = StructType([
    StructField("rank", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("forbes_id", StringType(), True),
    StructField("net_worth", IntegerType(), True),
    StructField("age4", FloatType(), True),
    StructField("age_range", StringType(), True),
    StructField("country", StringType(), True),
    StructField("source", StringType(), True),
    StructField("industry", StringType(), True),
    StructField("Age9", StringType(), True),
    StructField("Source of Wealth", StringType(), True),
    StructField("Self-Made Score", StringType(), True),
    StructField("Philanthropy Score", StringType(), True),
    StructField("Residence", StringType(), True),
    StructField("Citizenship", StringType(), True),
    StructField("Marital Status", StringType(), True),
    StructField("Children", StringType(), True),
    StructField("Education", StringType(), True),
    StructField("Bachelor", StringType(), True),
    StructField("Master", StringType(), True),
    StructField("Doctorate", StringType(), True),
    StructField("Drop Out", StringType(), True),
    StructField("Self Made", StringType(), True)
])

In [0]:
df=read_file('csv',"dbfs:/FileStore/tables/forbes_2640_billionaires.csv",header=True,schema=schema)

In [0]:
summary_df(df).display()

summary,rank,name,forbes_id,net_worth,age4,age_range,country,source,industry,Age9,Source of Wealth,Self-Made Score,Philanthropy Score,Residence,Citizenship,Marital Status,Children,Education,Bachelor,Master,Doctorate,Drop Out,Self Made
count,2640,2640,2640,2640,2575,2640,2640,2640,2640,2555,2639,558,411,2581,2629,2089,1598,1415,1416,1416,1416,1416,2640
mean,1289.159090909091,null,null,4.623787878787917,65.13864077669903,6.782954545454546,null,null,null,65.45714285714286,null,6.586021505376344,1.927007299270073,null,null,null,2.969336670838548,null,0.6864406779661016,0.3700564971751412,0.03672316384180791,0.06002824858757062,0.6863636363636364
stddev,739.6937255187028,null,null,9.834240939420306,13.258934406133013,1.7223573817784346,null,null,null,13.245203578351383,null,2.6445197450128655,1.021488892980818,null,null,null,1.67517090350861,null,0.4641034222933681,0.48299009455141895,0.18814774188797712,0.23762309269889065,0.4640583660212419
min,1,A. Jayson Adair,a-jayson-adair,1.0,101.0,0,Algeria,3D printing,Automotive,100.0,"3D printing, Self Made",1.0,1.0,"A Coruña, Spain",Algeria,Divorced,1.0,"Associate in Arts/Science, Brevard Community College; Bachelor of Arts/Science, Southern College of Optometry; Doctorate, Southern College of Optometry",0.0,0.0,0.0,0.0,0
25%,659.0,null,null,1.5,56.0,6.0,null,null,null,56.0,null,5.0,1.0,null,null,null,2.0,null,0.0,0.0,0.0,0.0,0.0
50%,1312.0,null,null,2.3,65.0,7.0,null,null,null,66.0,null,8.0,2.0,null,null,null,3.0,null,1.0,0.0,0.0,0.0,1.0
75%,1905.0,null,null,4.2,75.0,8.0,null,null,null,75.0,null,8.0,2.0,null,null,null,4.0,null,1.0,1.0,0.0,0.0,1.0
max,99,Zygmunt Solorz-Zak,zygmunt-solorz-zak,94.5,99.0,9,Zimbabwe,software services,Telecom,99.0,"waste management, Self Made",9.0,5.0,"torino, Italy",Zimbabwe,"Widowed, Remarried",9.0,error,1.0,1.0,1.0,1.0,1


In [0]:
df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- forbes_id: string (nullable = true)
 |-- net_worth: integer (nullable = true)
 |-- age4: float (nullable = true)
 |-- age_range: string (nullable = true)
 |-- country: string (nullable = true)
 |-- source: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- Age9: string (nullable = true)
 |-- Source of Wealth: string (nullable = true)
 |-- Self-Made Score: string (nullable = true)
 |-- Philanthropy Score: string (nullable = true)
 |-- Residence: string (nullable = true)
 |-- Citizenship: string (nullable = true)
 |-- Marital Status: string (nullable = true)
 |-- Children: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Bachelor: string (nullable = true)
 |-- Master: string (nullable = true)
 |-- Doctorate: string (nullable = true)
 |-- Drop Out: string (nullable = true)
 |-- Self Made: string (nullable = true)



In [0]:
df.summary().display()

summary,rank,name,forbes_id,net_worth,age4,age_range,country,source,industry,Age9,Source of Wealth,Self-Made Score,Philanthropy Score,Residence,Citizenship,Marital Status,Children,Education,Bachelor,Master,Doctorate,Drop Out,Self Made
count,2640,2640,2640,0,2575,2640,2640,2640,2640,2555,2639,558,411,2581,2629,2089,1598,1415,1416,1416,1416,1416,2640
mean,1289.159090909091,null,null,null,65.13864077669903,6.782954545454546,null,null,null,65.45714285714286,null,6.586021505376344,1.927007299270073,null,null,null,2.969336670838548,null,0.6864406779661016,0.3700564971751412,0.03672316384180791,0.06002824858757062,0.6863636363636364
stddev,739.6937255187028,null,null,null,13.258934406133013,1.7223573817784346,null,null,null,13.245203578351383,null,2.6445197450128655,1.021488892980818,null,null,null,1.67517090350861,null,0.4641034222933681,0.48299009455141895,0.18814774188797712,0.23762309269889065,0.4640583660212419
min,1,A. Jayson Adair,a-jayson-adair,null,18.0,0,Algeria,3D printing,Automotive,100.0,"3D printing, Self Made",1.0,1.0,"A Coruña, Spain",Algeria,Divorced,1.0,"Associate in Arts/Science, Brevard Community College; Bachelor of Arts/Science, Southern College of Optometry; Doctorate, Southern College of Optometry",0.0,0.0,0.0,0.0,0
25%,659,null,null,null,56.0,6.0,null,null,null,56.0,null,5.0,1.0,null,null,null,2.0,null,0.0,0.0,0.0,0.0,0.0
50%,1312,null,null,null,65.0,7.0,null,null,null,66.0,null,8.0,2.0,null,null,null,3.0,null,1.0,0.0,0.0,0.0,1.0
75%,1905,null,null,null,75.0,8.0,null,null,null,75.0,null,8.0,2.0,null,null,null,4.0,null,1.0,1.0,0.0,0.0,1.0
max,2540,Zygmunt Solorz-Zak,zygmunt-solorz-zak,null,101.0,9,Zimbabwe,software services,Telecom,99.0,"waste management, Self Made",9.0,5.0,"torino, Italy",Zimbabwe,"Widowed, Remarried",9.0,error,1.0,1.0,1.0,1.0,1


In [0]:
null_per(df).display()

Column,Null_Percentage
Philanthropy Score,84.43181818181819 %
Self-Made Score,78.86363636363636 %
Education,46.40151515151515 %
Drop Out,46.36363636363637 %
Doctorate,46.36363636363637 %
Master,46.36363636363637 %
Bachelor,46.36363636363637 %
Children,39.46969696969697 %
Age9,3.2196969696969697 %
Marital Status,20.87121212121212 %


In [0]:
print(df.columns)

['rank', 'name', 'forbes_id', 'net_worth', 'age4', 'age_range', 'country', 'source', 'industry', 'Age9', 'Source of Wealth', 'Self-Made Score', 'Philanthropy Score', 'Residence', 'Citizenship', 'Marital Status', 'Children', 'Education', 'Bachelor', 'Master', 'Doctorate', 'Drop Out', 'Self Made']


In [0]:
unique_values(df).display()

Column,UniqueValuesCount
Self Made,2
Bachelor,3
Drop Out,3
Doctorate,3
Master,3
Philanthropy Score,6
Marital Status,9
age_range,11
Self-Made Score,11
Children,16


In [0]:
count_of_unique_value(df,'country').display()

country,Count
United States,735
China,494
India,169
Germany,126
Russia,104
Hong Kong,66
Italy,64
Canada,63
Taiwan,52
United Kingdom,52


In [0]:
count_of_unique_value(df,'industry').display()

industry,Count
Finance & Investments,372
Manufacturing,324
Technology,314
Fashion & Retail,266
Food & Beverage,212
Healthcare,201
Real Estate,193
Diversified,187
Energy,100
Media & Entertainment,91


In [0]:
df_1=read_file('csv',"dbfs:/FileStore/tables/sample_data_for_data_profiling.csv",header=True)

In [0]:
df_1.display()

id,first_name,last_name,gender,age,education,occupation,income,street_address,city,state
1,Madaline,Murphy,female,24,Doctoral,Botanist,116989,384-7572 Risus Ave,New York City,New York
2,Kristian,Stewart,Male,24,Upper secondary,Salesman,45736,Ap #139-5202 Donec Ave,Los Angeles,null
3,Aston,Baker,Male,18,Master,Photographer,185036,Ap #536-5718 Orci Rd.,Chicago,Illinois
4,Brooke,Holmes,Female,30,Primary,Social Worker,66673,"P.O. Box 961, 1525 Eu Rd.",Houston,Texas
5,Jenna,Crawford,Female,28,Doctoral,Actor,93689,"P.O. Box 940, 7197 Cursus, St.",Phoenix,null
6,Dale,Williams,Male,24,Primary,Dancer,183076,"P.O. Box 351, 3083 Nulla Ave",null,Pennsylvania
7,Garry,Hall,Male,20,Master,Actor,75410,"P.O. Box 659, 6221 Ligula. Road",San Antonio,Texas
8,Darcy,Barnes,Female,30,Doctoral,Lawer,155934,"P.O. Box 375, 3299 Velit. Av.",San Diego,California
9,Reid,Sullivan,Male,22,Master,Astronomer,55463,"P.O. Box 351, 860 A, Rd.",Dallas,Texas
10,Daniel,Bailey,Male,null,Master,Singer,122878,5847 Eget Av.,Austin,Texas


In [0]:
summary_df(df_1).display()

summary,id,first_name,last_name,gender,age,education,occupation,income,street_address,city,state
count,40,40,40,40,32,40,40,40,40,33,38
mean,20.5,null,null,null,26.3125,null,null,117556.55,null,null,null
stddev,11.69045194450012,null,null,null,26.182285934849023,null,null,47448.1794122377,null,null,null
min,1,Adam,Bailey,Female,-18,Bachelor,Accountant,100969,2121 Sollicitudin Rd.,Atlanta,Arizona
25%,10.0,null,null,null,20.0,null,null,74585.0,null,null,null
50%,20.0,null,null,null,25.0,null,null,116989.0,null,null,null
75%,30.0,null,null,null,28.0,null,null,156289.0,null,null,null
max,9,Ted,Wright,female,30,Upper secondary,Social Worker,94434,"P.O. Box 988, 2100 Dui, Avenue",Tucson,Wisconsin


In [0]:
null_per(df_1).display()


Column,Null_Percentage
state,5.0 %
age,20.0 %
city,17.5 %
street_address,0.0 %
income,0.0 %
occupation,0.0 %
education,0.0 %
id,0.0 %
gender,0.0 %
last_name,0.0 %


In [0]:
unique_values(df_1).display()


Column,UniqueValuesCount
gender,4
education,6
age,16
state,26
occupation,27
city,34
last_name,37
first_name,38
street_address,40
income,40


In [0]:
count_of_unique_value(df_1,'state').display()

state,Count
Texas,6
California,5
null,2
North Carolina,2
Arizona,2
Tennessee,2
Colorado,2
Ohio,1
Oregon,1
Pennsylvania,1


In [0]:
count_of_unique_value(df_1,'city').display()

city,Count
null,7
Phoenix,1
Dallas,1
San Antonio,1
Louisville,1
Los Angeles,1
Sacramento,1
San Diego,1
San Francisco,1
Nashville,1


In [0]:
regex = {'age': '^[0-99]{1,2}$', 'first_name': '^[a-zA-Z]*$', 'gender': '^M(ale)?$|^F(emale)?$'}
miss_matched(df,regex).display()

Column,MismatchPercentage
age,7.5 %
gender,7.5 %
first_name,0.0 %


3
